# Computational Phase Retrieval with Tensor Methods

## Device Information

In [1]:
!nvidia-smi

Fri Jul  9 01:55:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8     5W /  N/A |    293MiB /  3911MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Import Required Libraries

In [5]:
import tensorflow as tf
print(f"Tensorflow version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        print("Num GPUs Available: ", len(gpus))
        for gpu in gpus:
            # Allow memory growth for the GPU.
            # Reference: https://www.tensorflow.org/guide/gpu
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized.
        print(e)

from matplotlib import pyplot as plt
plt.style.use('dark_background')
import numpy as np
import tensorly as tl
import cv2
import time
import os

import matplotlib.cm as cm
import seaborn as sns

from matplotlib import rc, rcParams
rcParams['font.family'] = 'Cubano'
rc('text', usetex=False)

Tensorflow version: 2.5.0
Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


## GPU Benchmark

Reference: https://www.tensorflow.org/guide/gpu

In [6]:
tf.debugging.set_log_device_placement(True)

n = 5000
num_iters = 10

'''
Test with TensorFlow GPU.
'''
start_tf = time.time()

for i in range(num_iters):
    # Tensors are defaultly placed on the GPU (CPU would be considerably slower due
    # to the incurred communication cost).
    # with tf.device('/CPU:0'):
    a = tf.ones((n, n))
    b = tf.ones((n, n))

    # Run on the GPU
    c = tf.matmul(a, b)

print(f'Elapsed time with TensorFlow GPU: {time.time() - start_tf}') # about 0.02 seconds

'''
Test with Numpy.
'''
start_np = time.time()

for i in range(num_iters):
    a = np.ones((n, n))
    b = np.ones((n, n))

    c = np.dot(a, b)

print(f'Elapsed time with Numpy: {time.time() - start_np}') # about 15 seconds


Elapsed time with TensorFlow GPU: 0.5843789577484131
Elapsed time with Numpy: 18.500978231430054


## Low Rank Phase Retrieval

References:

\[1\] Namrata Vaswani, Seyedehsara Nayer, Yonina C. Eldar. *Low Rank Phase Retrieval*. https://rutgers.box.com/s/dntl0sh157p62rgi1zerdaxrqthugr32

\[2\] Namrata Vaswani. *Nonconvex Structured Phase Retrieval*. https://rutgers.box.com/s/x02w8frd1ep01cxdjlnojufa9npvstsz.

\[3\] Tamara G. Kolda, Brett W. Bader. *Tensor Decompositions and Applications*. https://rutgers.box.com/s/aq9psx3mgwhms6rrzlhn94h56c3oshox. 




### Define Data Directories

In [17]:
INPUT_DIR = './videos/' # directory of the test videos
OUTPUT_DIR = './output/' # output directory
FRAMES_DIR = './ouput/frames/' # output directory of the extracted video frames 


In [20]:
# Read the video.
video_path = INPUT_DIR + os.listdir(INPUT_DIR)[0] # define video path
cap = cv2.VideoCapture(video_path) # read the video from path
video_name = os.listdir(INPUT_DIR)[0].split('.')[0] # get the name of the video

# Creat the folder to store the extracted frames of the video.
try:
    if not os.path.exists(FRAMES_DIR + video_name):
        os.makedirs(FRAMES_DIR + video_name)
    else:
        print('Directory already exists!')
except OSError:
    print('OS ERROR')

k = 0 # frame number, k = 0, 1, 2, ..., q - 1
while (True):
    # Capture the video frame-by-frame.
    # Code adopted: https://docs.opencv.org/3.4/dd/d43
    # # tutorial_py_video_display.html
    ret, frame = cap.read()

    # If the frame is read correctly the return boolean (ret) is true.
    if not ret:
        print("Cannot receive frame (probably end of stream). Exiting...")
        break
    else:
        # Convert the frame to grayscale.
        # grayscale = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        name = FRAMES_DIR + cap + f'/frame-{k}.jpg'
        print('DEBUG: Captured...' + name)
        cv2.imwrite(name, frame)
        k += 1

# Release the capture when finished.
cap.release()
cv2.destroyAllWindows()